In [ ]:
import re
import pandas as pd
import numpy as np
import ast
from collections import defaultdict
from datetime import datetime, timedelta

## Text Cleaning

In [2]:
df = pd.read_csv("detailed_job_infos.csv")

### Job Description

In [3]:
def clean_job_description(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # Step 1: Replace problematic unicode spaces
    text = text.replace('\u202f', ' ')   # Narrow no-break space
    text = text.replace('\xa0', ' ')     # Non-breaking space
    text = text.replace('\r', '')        # Carriage return

    # Step 2: Collapse multiple newlines to 2 (for paragraphs)
    text = re.sub(r'\n{3,}', '\n\n', text)

    # Step 3: Remove line breaks that split sentences or phrases
    text = re.sub(r'(?<=[a-z0-9])\n(?=[a-z0-9])', ' ', text)
    text = re.sub(r'(?<=[A-Za-z0-9])\n(?=[A-Za-z0-9])', ' ', text)

    # Step 4: Standardize double newlines (if you want real paragraphs)
    text = re.sub(r'\n{2,}', '\n\n', text)   # Keep \n\n as paragraph separator

    # Step 5: Replace remaining single \n with space (not paragraphs)
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Step 6: Normalize spacing
    text = re.sub(r'[ \t]+', ' ', text)
    text = text.strip()

    return text

In [4]:
df["job_description"] = df["job_description"].apply(clean_job_description)

In [5]:
def expand_job_description_meta(df, meta_column="job_description_meta"):
    """
    Parse and expand a stringified dict column (like 'job_description_meta') into new columns.

    Args:
        df (pd.DataFrame): Input DataFrame with a column of stringified dicts.
        meta_column (str): Column name containing stringified dictionaries.

    Returns:
        pd.DataFrame: Original DataFrame with new columns extracted from the dict.
    """
    def safe_parse(s):
        try:
            return ast.literal_eval(s) if isinstance(s, str) else {}
        except:
            return {}

    # Parse and expand
    meta_parsed = df[meta_column].apply(safe_parse)
    meta_df = pd.DataFrame(meta_parsed.tolist())

    # Merge and return
    df_expanded = pd.concat([df, meta_df], axis=1)
    return df_expanded

In [6]:
df = expand_job_description_meta(df)

### Location and State

In [7]:
add = pd.read_csv("additional_infos.csv")
additional_info = add[['job_url', 'workplace_type']]

In [8]:
df_merged = pd.merge(df, additional_info, on="job_url", how="left")

In [9]:
def extract_state(location):
    if pd.isna(location):
        return "Unknown"
    if "United States" in location:
        return "Unknown"
    
    # Match ", XX" where XX is 2 uppercase letters (state abbreviation)
    match = re.search(r",\s*([A-Z]{2})$", location.strip())
    if match:
        return match.group(1)
    return "Unknown"

# Apply to the merged DataFrame
df_merged["State"] = df_merged["company_location"].apply(extract_state)

In [10]:
df_merged.drop(["company_location", "job_description_meta"], axis=1, inplace=True)

### Salary

In [16]:
def parse_salary(s):
    """
    Parse salary string into an estimated annual salary (numeric).
    Handles hourly, weekly, monthly, yearly, and ranges.
    Assumptions:
    - Hourly: 40 hrs/week, 52 weeks/year
    - Weekly: 52 weeks/year
    - Monthly: 12 months/year
    """
    if not isinstance(s, str) or not s.strip():
        return np.nan
    
    s = s.lower().replace(",", "").strip()

    # Extract numbers (with optional K or M)
    nums = re.findall(r"\$?\d+(?:\.\d+)?[kKmM]?", s)
    if not nums:
        return np.nan
    
    # Convert numbers into floats
    values = []
    for n in nums:
        n = n.replace("$", "")
        if "k" in n:
            values.append(float(n[:-1]) * 1000)
        elif "m" in n:
            values.append(float(n[:-1]) * 1_000_000)
        else:
            values.append(float(n))
    
    # If it's a range, take the average
    val = sum(values) / len(values)

    # Detect time unit
    if re.search(r"/?\s*hour|/hr", s):
        val *= 40 * 52
    elif re.search(r"/?\s*week|/wk", s):
        val *= 52
    elif re.search(r"/?\s*month|/mo", s):
        val *= 12
    elif re.search(r"/?\s*year|/yr|/y", s):
        pass  # already annual
    else:
        # No explicit unit → assume yearly
        pass

    return val

In [26]:
df_merged["salary_range"] = df["salary_range"].apply(parse_salary)
df_merged.rename(columns={"salary_range": "Salary ($)"}, inplace=True)

### Publish Time

In [33]:
def convert_relative_to_date(df, col_name="publish_time", new_col="publish_date"):
    def parse_relative_time(text):
        # Skip NaN or non-string
        if not isinstance(text, str):
            return None

        text = text.lower().strip()
        today = datetime.today()

        match = re.match(r"(\d+)\s+(day|week|month|year)s?\s+ago", text)
        if not match:
            return None

        value, unit = int(match.group(1)), match.group(2)

        if unit == "day":
            delta = timedelta(days=value)
        elif unit == "week":
            delta = timedelta(weeks=value)
        elif unit == "month":
            delta = timedelta(days=30 * value)
        elif unit == "year":
            delta = timedelta(days=365 * value)
        else:
            return None

        return (today - delta).strftime("%m/%d/%y")  # Format as MM/DD/YY

    df[new_col] = df[col_name].apply(parse_relative_time)
    return df

In [37]:
df_merged = convert_relative_to_date(df_merged)
df_merged.drop(["publish_time"], axis =1, inplace = True)

In [39]:
df_merged.to_csv("../linkedin_data_center_jobs.csv", index = False)